In [30]:
import numpy as np
import pandas as pd
from dotenv import load_dotenv
import requests
import os
import csv
from datetime import datetime


Data

In [54]:

load_dotenv()
api_key = os.getenv('ATTOM_API')
headers = {
    'apikey': api_key 
}

def call_attom_api(api_version, endpoint, route, prefix, query_params=None):
    base_url = "https://api.gateway.attomdata.com" 
    url = f"{base_url}/{endpoint}/{api_version}/{prefix}/{route}"

    response = requests.get(url, headers=headers, params=query_params)

    if response.status_code == 200:
        return response.json()  
    else:
        response.raise_for_status()  

def flatten_json(json_obj, parent_key='', sep='.'):
    items = []
    for k, v in json_obj.items():
        new_key = f"{parent_key}{sep}{k}" if parent_key else k
        if isinstance(v, dict):
            items.extend(flatten_json(v, new_key, sep=sep).items())
        elif isinstance(v, list):
            for i, elem in enumerate(v):
                items.extend(flatten_json(elem, f"{new_key}_{i}", sep=sep).items())
        else:
            items.append((new_key, v))
    return dict(items)


api_version = "v1.0.0"  
endpoint = "propertyapi" 
route = "basicprofile"  
prefix = "property"
query_params = {
    'pagesize' : 2000,
    'address1' : "4529 Winona Court",
    'address2' : "Denver, CO", 
    'radius' : 5.0
    
}

try:
    result = call_attom_api(api_version, endpoint, route, prefix, query_params)
    
except requests.exceptions.HTTPError as e:
    print(f"HTTP error occurred: {e}")
except Exception as e:
    print(f"An error occurred: {e}")



In [56]:
headers = [
    'saleAmt', 'lotSize1', 'lotSize2',
    'yearBuilt', 'bldgSize', 'livingSize', 'roomsTotal', 'beds', 'bathsTotal', 'bsmtSize', 'condition',
    'latitude', 'longitude', 'distance',
    'saleTransType', 'saleRecDate',
    'mktTtlValue', 'assdTtlValue',
    'taxAmt', 'taxYear'
]

rows = []
for property_data in result['property']:
    row = {
        'saleAmt': property_data.get('sale', {}).get('saleAmountData', {}).get('saleAmt', ''),
        'lotSize1': property_data.get('lot', {}).get('lotSize1', ''),
        'lotSize2': property_data.get('lot', {}).get('lotSize2', ''),
        'yearBuilt': property_data.get('summary', {}).get('yearBuilt', ''),
        'bldgSize': property_data.get('building', {}).get('size', {}).get('bldgSize', ''),
        'livingSize': property_data.get('building', {}).get('size', {}).get('livingSize', ''),
        'roomsTotal': property_data.get('building', {}).get('rooms', {}).get('roomsTotal', ''),
        'beds': property_data.get('building', {}).get('rooms', {}).get('beds', ''),
        'bathsTotal': property_data.get('building', {}).get('rooms', {}).get('bathsTotal', ''),
        'bsmtSize': property_data.get('building', {}).get('rooms', {}).get('bsmtSize', ''),
        'condition': property_data.get('building', {}).get('construction', {}).get('condition', ''),
        'latitude': property_data.get('location', {}).get('latitude', ''),
        'longitude': property_data.get('location', {}).get('longitude', ''),
        'distance': property_data.get('location', {}).get('distance', ''),
        'saleTransType': property_data.get('sale', {}).get('saleAmountData', {}).get('saleTransType', ''),
        'saleRecDate': property_data.get('sale', {}).get('saleAmountData', {}).get('saleRecDate', ''),
        'mktTtlValue': property_data.get('assessment', {}).get('market', {}).get('mktTtlValue', ''),
        'assdTtlValue': property_data.get('assessment', {}).get('assessed', {}).get('assdTtlValue', ''),
        'taxAmt': property_data.get('tax', {}).get('taxAmt', ''),
        'taxYear': property_data.get('tax', {}).get('taxYear', '')
    }
    rows.append(row)
with open('property_data.csv', 'w', newline='') as file:
    writer = csv.DictWriter(file, fieldnames=headers)
    writer.writeheader()
    writer.writerows(rows)

print("CSV file created successfully.")

CSV file created successfully.


In [57]:
def days_from_today(sale_date):
    today = datetime.today()
    sale_date = datetime.strptime(sale_date, '%Y-%m-%d')  
    return (today - sale_date).days

sales_trans_type_mapping = {
    "Resale": 0,
    "New construction": 1,
    "Nominal/Quit Claim": 2,
    "Construction Loan/Financing": 3
}

condition_mapping = {
    "GOOD": 1,
    "AVERAGE": 0
}

def process_property_data(properties):
    rows = []
    
    for property_data in properties:
        sale_amt = property_data.get('sale', {}).get('saleAmountData', {}).get('saleAmt')
        if sale_amt is None:
            continue
        
        # Extract and transform other features
        lot_size1 = property_data.get('lot', {}).get('lotSize1')
        lot_size2 = property_data.get('lot', {}).get('lotSize2')
        year_built = property_data.get('building', {}).get('yearBuilt')
        bldg_size = property_data.get('building', {}).get('size', {}).get('bldgSize')
        living_size = property_data.get('building', {}).get('size', {}).get('livingSize')
        rooms_total = property_data.get('building', {}).get('rooms', {}).get('roomsTotal')
        beds = property_data.get('building', {}).get('rooms', {}).get('beds')
        baths_total = property_data.get('building', {}).get('rooms', {}).get('bathsTotal')
        bsmt_size = property_data.get('building', {}).get('interior', {}).get('bsmtSize')
        
        condition = condition_mapping.get(property_data.get('building', {}).get('construction', {}).get('condition'))
        latitude = property_data.get('location', {}).get('latitude')
        longitude = property_data.get('location', {}).get('longitude')
        distance = property_data.get('location', {}).get('distance')
        
        # Process non-numerical fields
        sale_trans_type = sales_trans_type_mapping.get(property_data.get('sale', {}).get('saleAmountData', {}).get('saleTransType'))
        sale_rec_date = days_from_today(property_data.get('sale', {}).get('saleAmountData', {}).get('saleRecDate'))
        
        mkt_ttl_value = property_data.get('assessment', {}).get('market', {}).get('mktTtlValue')
        assd_ttl_value = property_data.get('assessment', {}).get('assessed', {}).get('assdTtlValue')
        tax_amt = property_data.get('assessment', {}).get('tax', {}).get('taxAmt')
        tax_year = property_data.get('assessment', {}).get('tax', {}).get('taxYear')
        
        # Add to row
        rows.append([
            sale_amt, lot_size1, lot_size2, year_built, bldg_size, living_size, rooms_total, beds, 
            baths_total, bsmt_size, condition, latitude, longitude, distance, sale_trans_type, 
            sale_rec_date, mkt_ttl_value, assd_ttl_value, tax_amt, tax_year
        ])
    
    return rows

def write_to_csv(filename, rows):
    header = ['saleAmt', 'lotSize1', 'lotSize2', 'yearBuilt', 'bldgSize', 'livingSize', 
              'roomsTotal', 'beds', 'bathsTotal', 'bsmtSize', 'condition', 'latitude', 
              'longitude', 'distance', 'saleTransType', 'saleRecDate', 'mktTtlValue', 
              'assdTtlValue', 'taxAmt', 'taxYear']
    
    with open(filename, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(header)
        writer.writerows(rows)

processed_data = process_property_data(properties)
write_to_csv('real_estate_data.csv', processed_data)

AttributeError: 'str' object has no attribute 'get'